<a href="https://colab.research.google.com/github/jazzjastine/Natural_Language_Processing/blob/main/Mini_Corpus_Programming_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
!pip install pdfplumber gensim spacy
!python -m spacy download en_core_web_sm

import pdfplumber
import spacy
from gensim import corpora, models
from gensim.models.phrases import Phrases, Phraser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 39.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [30]:
# Initialize spaCy
nlp = spacy.load("en_core_web_sm")

In [31]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    all_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                all_text += ' ' + text
    return all_text

In [32]:
# Function to Preprocess Text with spaCy
def preprocess_text(document):
    # Parse the document with spaCy
    doc = nlp(document)
    # Filter out punctuation, stopwords, and lemmatize
    return [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.like_num]

In [33]:
# Google drive folder
folder_path = '/content/drive/My Drive/NLP_PDFs'

# Example list of PDF filenames
pdf_filenames = ["sustainability-11-02738.pdf", "PIIS2405844019358414.pdf", "kundzewicz-et-al-2018-flood-risk-reduction-structural-measures-and-diverse-strategies.pdf", "Academic Emergency Medicine - 2008 - Lai - Strategies of Disaster Response in the Health Care System for Tropical Cyclones .pdf", "1-s2.0-S2225603221000138-main.pdf", "1-s2.0-S2225603219300475-main.pdf"]

In [48]:
# Extract and preprocess text from PDFs
processed_texts = []
for filename in pdf_filenames:
    pdf_path = f"{folder_path}/{filename}"
    text = extract_text_from_pdf(pdf_path)
    processed_text = preprocess_text(text)
    processed_texts.append(processed_text)

In [50]:
# 1. Preprocess and Vectorization using Gensim
dictionary = corpora.Dictionary(processed_texts)

In [65]:
# 2. Document to Bag-of-Words
corpus = [dictionary.doc2bow(text) for text in processed_texts]

In [66]:
# Storing your generated corpus
corpora.MmCorpus.serialize('disaster_corpus.mm', corpus)

In [67]:
# 3. Bag-of-Words to TF-IDF Representation
tfidf_model = models.TfidfModel(corpus)
corpus_tfidf = tfidf_model[corpus]

In [68]:
# 4. N-Gramming
bigram_phrases = Phrases(processed_texts, min_count=3, threshold=10)
bigram = Phraser(bigram_phrases)
texts_with_bigrams = [bigram[text] for text in processed_texts]

In [69]:
# Update dictionary and corpus to include bigrams
dictionary.add_documents(texts_with_bigrams)

In [70]:
# Filter out tokens that appear too frequently or too infrequently
dictionary.filter_extremes(no_below=2, no_above=0.9)

In [71]:
# Re-create corpus after filtering
corpus_filtered = [dictionary.doc2bow(text) for text in texts_with_bigrams]

In [74]:
print("First Document's Original Tokens:", processed_texts[0])
print("\nFirst Document's Bag-of-Words Representation:", corpus[0])
print("\nFirst Document's TF-IDF Representation:", list(corpus_tfidf)[0])
print("\nFirst Document with Bigrams:", texts_with_bigrams[0])
print("\nDocument to Bag-of-Words Representation after filtering:", corpus_filtered[0])
print("\nDictionary after filtering extremes and adding bigrams:", dictionary.token2id)

First Document's Original Tokens: [' ', 'sustainability', '\n', 'article', '\n', 'risk', 'Assessment', 'Flood', 'Disaster', 'Induced', '\n', 'Typhoon', 'Rainstorms', 'Guangdong', 'Province', 'China', '\n', 'JiayangZhangandYangboChen', '\n', 'schoolofgeographyandplanne', 'SunYat', 'SenUniversity', 'Guangzhou510245,China', '\n', 'zhangjy225@mail2.sysu.edu.cn', '\n', 'correspondence', 'eescyb@mail.sysu.edu.cn;Tel.:+86', '\n', 'cid:1)(cid:2)(cid:3)(cid:1)(cid:4)(cid:5)(cid:6)(cid:7)(cid:8)(cid:1', '\n', 'cid:1)(cid:2)(cid:3)(cid:4)(cid:5)(cid:6)(cid:7', '\n', 'receive', '21march2019;accepted', '8may2019;published', '14may2019', '\n', 'Abstract', 'China', 'coastal', 'area', 'suffer', 'typhoon', 'attack', 'year', 'rainstorm', 'induce', '\n', 'typhoonscharacteristicallyarehighintensitywithalargeamountofrainandusuallyinduceflood', '\n', 'andwaterloggingintheaffectedarea', 'guangdongprovincehasthehighestfrequencyoftyphoon', '\n', 'hit', 'China', 'special', 'geographical', 'position', 'unique', 